<a href="https://colab.research.google.com/github/Mollylst/DeepFake-Project/blob/main/resize%2C_mean_and_std.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Resize images from 1024x1024 to 128x128


In [ ]:
from PIL import Image
import glob
import os

# new folder path (may need to alter for Windows OS)
# change path to your path
path = '/content/drive/MyDrive/real_fake_20000/train/real' #the path where to save resized images
# create new folder
if not os.path.exists(path):
    os.makedirs(path)

# loop over existing images and resize
# change path to your path
for filename in glob.glob('/content/drive/MyDrive/real_fake_20000/train/real/*.png'): #path of raw images
    img = Image.open(filename).resize((128,128))
    # save resized images to new folder with existing filename
    img.save('{}{}{}'.format(path,'/',os.path.split(filename)[1]))

Upload the zip file to colab local file

Create a file named train in colab local

In [1]:
!unzip -uq '/content/drive/MyDrive/FFHQ data 8:1:1/train-fake-stylegan1.zip' -d /content/dataset
!unzip -uq '/content/drive/MyDrive/FFHQ data 8:1:1/train_real.zip' -d /content/dataset

Mean and Standard

In [2]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from tqdm import tqdm
import numpy as np


Batchsize=128
data_dir='/content/dataset'
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_set = datasets.ImageFolder(data_dir, transform=transforms.ToTensor())
#train_set = datasets.ImageFolder(data_dir+'/train', transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=Batchsize, shuffle=True)

def get_mean_std(loader):
    # var[X] = E[X**2] - E[X]**2
    channels_sum, channels_sqrd_sum, num_batches = 0, 0, 0

    for data, _ in tqdm(loader):
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_sqrd_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_sqrd_sum / num_batches - mean ** 2) ** 0.5

    return mean, std


mean, std = get_mean_std(train_loader)
print(mean)
print(std)

100%|██████████| 141/141 [00:30<00:00,  4.67it/s]


tensor([0.5184, 0.4233, 0.3776])
tensor([0.2795, 0.2535, 0.2548])
